In [ ]:
pip install wikipedia-api

In [4]:
import wikipediaapi


wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName (merlin@example.com)',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [ ]:
from random import shuffle

def url_categorymembers(categorymembers, store, level=0, max_level=1):
    pages_cat = list(categorymembers.values())
    shuffle(pages_cat)
    i = 0 # nb de pages
    for c in pages_cat:
        if i==1000: #1000 pages par catégorie
            break
        if c.title[:6]!="Portal" and c.title[:8]!="Category":
            i+=1
            store.append(c.fullurl)
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            url_categorymembers(c.categorymembers, store, level=level + 1, max_level=max_level)
    return store


cat = wiki_wiki.page("Category:Science")
science = url_categorymembers(cat.categorymembers,[])